In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt

import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
from sklearn.preprocessing import StandardScaler#モデルの評価用に標準化する関数
import scipy.stats#モデルの評価用に標準化する関数
import warnings
#from pycaret.regression import *#setupが回帰と分類で名前が同じだが機能が違うので，使う方のみインポート（後にインポートした方が使われる)
from pycaret.classification import *#setupが回帰と分類で名前が同じだが機能が違うので，使う方のみインポート（後にインポートした方が使われる)
from imblearn.over_sampling import ADASYN, BorderlineSMOTE, KMeansSMOTE, RandomOverSampler, SMOTE, SMOTENC, SVMSMOTE#不均衡データに対する追加機能

#自作のモジュールのインポート
sys.path.append("../..")
import module.master as master
import module.modeling_scores as modeling


In [2]:
place_master=master.get_place_master()
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "../../../bot_database/{place_name}/model_score_{place_name}/".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass
    

In [3]:
version_big='v5'#バージョン(大のくくり)
version='v5_1'#バージョン

place_master=master.get_place_master()
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "../../../bot_database/{place_name}/model_score_{place_name}/".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass


#V4_系列の特殊処理
for place in place_master.items():
    #print(place[0],place[1],'\n')
    place_name=place[1]
    dir_path = "../../../bot_database/{place_name}/model_score_{place_name}/{v_b}_score/{version}/".format(place_name=place_name,v_b=version_big,version=version)#作成したデータの書き込み先#使用するデータの読み込み
    if os.path.exists(dir_path)==False:
        os.makedirs(dir_path)
    else:
        pass

## 関数内で使用されている関数

In [4]:
def get_event_info(result_base_df):
    df=result_base_df.copy()
    df['date']=pd.to_datetime(df['date'])#日付が文字列なのでdateを日付型に変換
    df['year']=df['date'].dt.year
    df['month']=df['date'].dt.month
    df['day']=df['date'].dt.day

    num_date=1
    num_date_arr=[]
    last_race_date=df['date'].values[0]#前レースの日付(処理開始時用にtarainのデータの一番初めのdateを仮に入力しておく)
    for index,row in df.iterrows():
        today_date=row['date']
        if today_date==last_race_date:#同じ日のレースだったらおなじレース日を配列に追加、次の日の日付を出力（ほぼ無操作みたいなもん）
            next_date=row['date'] + datetime.timedelta(days=1)#次の日
            num_date_arr.append(num_date)
        else:#日にちが変わった時
            if today_date==next_date:#想定していた日付（次の日のレース）だったら,レース日を一日足して、そのレース日番号を加算
                num_date+=1
                num_date_arr.append(num_date)
                last_race_date=row['date']#前回レース日を上書き
                #next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
                next_date=row['date'] + datetime.timedelta(days=1)#次の日
                #print(next_date)
            else:#想定していた日付でない(違う大会になった)場合はレース日をリセット
                num_date=1
                num_date_arr.append(num_date)
                last_race_date=row['date']#前回レース日を上書き
                #next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
                next_date=row['date'] + datetime.timedelta(days=1)#次の日
    df['num_date']=num_date_arr

    range_races=0#大会中の取得できたレースの数
    range_date=1#大会の開催日数
    range_date_arr=[]
    range_date_arr_2=[]#for文中で繰り返し上書きさせる用の配列
    last_race_date=df['date'].values[0]#前レースの日付(処理開始時用にtrainのデータの一番初めのdateを仮に入力しておく)
    for index,row in df.iterrows():
        today_date=row['date']
        if today_date==last_race_date:#同じ日のレースだったらおなじレース日を配列に追加、次の日の日付を出力（ほぼ無操作みたいなもん）
            range_races+=1
            next_date=row['date'] + datetime.timedelta(days=1)#次の日
            #num_date_arr.append(num_date)
        else:#日にちが変わった時
            if today_date==next_date:#想定していた日付（次の日のレース）だったら,レース日を一日足して終了
                range_date+=1
                range_races+=1
                last_race_date=row['date']#前回レース日を上書き
                #next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
                next_date=row['date'] + datetime.timedelta(days=1)#次の日次の日
            else:#想定していた日付でない(違う大会になった)場合は現在のrange_dateをもとに前の大会のレースに大会開催日数を持たせる。

                range_date_arr_2=[range_date]*range_races
                for num in range_date_arr_2:
                    range_date_arr.append(num)
                range_races=1#大会中の取得できたレースの数
                range_date=1#大会の開催日数
                last_race_date=row['date']#前回レース日を上書き
                #next_date=train_df[train_df['date']==row['date'] + datetime.timedelta(days=1)]#次の日
                next_date=row['date'] + datetime.timedelta(days=1)#次の日
    range_date_arr_2=[range_date]*range_races#最後の日は日付の変わり絵が発生しないので特別処理
    for num in range_date_arr_2:
        range_date_arr.append(num)
    df['range_date']=range_date_arr

    #四半期カラムの作成
    df['season']=df['month']
    df['season']=df['season'].replace([3,4,5],'sp')#春
    df['season']=df['season'].replace([6,7,8],'su')#夏
    df['season']=df['season'].replace([9,10,11],'au')#秋
    df['season']=df['season'].replace([12,1,2],'wi')#冬
    #df=df.drop('date',axis=1)
    return df

def pred_th_trans(pred_df,th):
    #引数として予測結果のdeと、変換したい閾値を渡す。
    trans_df=pred_df.copy()
    trans_df.loc[trans_df['pred_proba'] >= th, 'pred'] = 1
    trans_df.loc[~(trans_df['pred_proba']  >=  th), 'pred'] = 0
    return trans_df

def calc_monthly_analysis(pred_df):#予測に加えて，配当，開催情報が結合されたdfを渡すことで月ごと関連の分析を行ってくれる関数
    cols=['month','use','get','income','income_per','num_hit','buy_hit_per','mean_income','median_income']
    monthly_analysis_df= pd.DataFrame(columns=cols)#月別収益結果の入る箱
    months=pred_df['month'].value_counts(sort=False).index
    for month in months:
        monthly_df=pred_df[pred_df['month']==month].copy()
        use_m=100*monthly_df['pred'].sum()
        get_m=monthly_df['gain'].sum()
        income=get_m-use_m
        income_per=(get_m/use_m)*100
        
        
        num_hit=monthly_df['hit_flag'].sum()
        num_pred=pred_df['pred'].sum()
        buy_hit_per=(num_hit/num_pred)*100
        if num_hit==0:#警告文削除用
            mean_income=0
            median_income=0
        else:
            mean_income=monthly_df[monthly_df['hit_flag']==1]["gain"].mean()#１回の的中あたりの平均配当
            median_income=monthly_df[monthly_df['hit_flag']==1]["gain"].median()#１回の的中あたりの中央配当
        
        append_arr=[month,use_m,get_m,income,income_per,num_hit,buy_hit_per,mean_income,median_income]
        append_s=pd.Series(append_arr,index=cols)
        monthly_analysis_df=monthly_analysis_df.append(append_s, ignore_index=True)
    return monthly_analysis_df

def get_season_date(now_date):#日付(datetime型)を渡すと，その日付で購買予測を行う際に使用するデータの区間を返す関数
    use_data_year=now_date.year
    if (now_date.month>=1)and(now_date.month<4):
        use_data_month=1
    elif (now_date.month>=4)and(now_date.month<7):
        use_data_month=4
    elif (now_date.month>=7)and(now_date.month<10):
        use_data_month=7
    elif (now_date.month>=10):
        use_data_month=10
    else:
        print('what???????')
    use_data_date=datetime.datetime(year=use_data_year, month=use_data_month,day=1)
    return use_data_date

def trans_result_com(target_com,trans_base_df):#comをターゲットに合わせて0,1の二値に変換する。
    #学習データのラベル変換==========================================================
    trans_df=trans_base_df.copy()
    #result_train_df=trans_base_df.copy()
    result_arr=[0]*len(trans_df)
    i=0
    for result in trans_df['result_com']:#
        if ((result==target_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1
    trans_df['result_com']=result_arr
    return trans_df


def pred_th_trans_com(pred_df,th,target_com):#指定の組のカラムのみを置換。
    trans_df=pred_df.copy()
    trans_df.loc[trans_df['pred_{}'.format(target_com)] >= th, 'pred_{}'.format(target_com)] = 1
    trans_df.loc[~(trans_df['pred_{}'.format(target_com)] >=  th), 'pred_{}'.format(target_com)] = 0
    return trans_df

def calc_gain(pred_gain_df):#レース単位であたっているか同課の判別と、当たった場合に得られた配当金を計算する関数
    pred_true_df=pred_gain_df[(pred_gain_df['pred']==1)&(pred_gain_df['trans_result']==1)].copy()
    pred_true_df['hit']=1
    calc_base_df=pred_gain_df.copy()
    calc_base_df['hit']=pred_true_df['hit']
    calc_base_df['gain']=pred_true_df['money']
    calc_base_df=calc_base_df.fillna(0)
    #
    #calc_base_df:予測、変換積みの結果、実際の結果、配当金、収益をすべて表したdf,合計操作は行っていない。
    #
    return calc_base_df

def check_pred_arr(pred1_df,pred2_df):#カラムの中身が同じか比較する関数
    pred_1_vals=[pred1_df[col] for col in pred1_df.columns]
    pred_2_vals=[pred2_df[col] for col in pred2_df.columns]
    for col_name1,col1,col_name2,col2 in zip(pred1_df.columns,pred_1_vals,pred2_df.columns,pred_2_vals):
        if list(col1.values)==list(col1.values):
            print(col_name1,'  and  ',col_name2,'  is same pred \n')
    return None

def pred_th_trans(pred_df,th):#閾値を渡して、その値以上を1、未満を0に置き変える。
    #引数として予測結果のdeと、変換したい閾値を渡す。
    trans_df=pred_df.copy()
    trans_df.loc[trans_df['pred_proba'] >= th, 'pred'] = 1
    trans_df.loc[~(trans_df['pred_proba']  >=  th), 'pred'] = 0
    return trans_df


## 前バージョンから持ってきた参考関数

In [5]:
def data_making_clustar_section_has_final(df,now_ym,range_test_m,range_final_m):#モデル関連に使用するdfの作成関数(クラスタリングあり、モータ番号、艇番号なし)（加工関数）final(実運用バックテスト区間)あり版
    #V4系列で使用する加工関数，年と月の情報を使って直近のデータを使って性能検証を行う（区間を使ってデータを作っている）
    #validデータを作成するバージョンなので実装する際はこれをそのまま使わず，final_test部分の処理は消してくだちい
    #yearが使わないと思うけど一応残してあるから邪魔だと思ったら消して下さい
    result_df=df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22}).copy()#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02}).copy()#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02}).copy()

    #result_base_df=result_df.copy()
    #result_base_df=get_event_info(result_base_df)#開催の情報について付与する関数(年月日に加えて，何日間の開催かどうかも教えてくれる)

    #ダミー変数化
    result_df_dummie=result_df.copy()
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1).copy()

    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr.copy()
        male_dummie_df=pd.get_dummies(result_df_dummie[col]).copy()#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val.copy()
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    cols=list(result_df_dummie.columns)



    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す

    #boat、moterの情報は使わない、
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()
    for col in boat_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    #クラスタリング
    #分けてみるクラスタの数は[3,5,7,9]の4個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    #学習データのdateを年に変換
    result_df_dummie['date']=pd.to_datetime(result_df_dummie['date'])#日付が文字列なのでdateを日付型に変換
    result_df_dummie['year']=result_df_dummie['date'].dt.year
    
    #==========================================================================
    #result_df_dummie=result_df_dummie[result_df_dummie['year']!=2020].copy()#2020のデータを完全に切り離す。
    #==========================================================================
    #クラアスタリング用の学習、予測用のデータの切り分け
    #年，月，日とかの取得
    #now_ym:開始の月
    test_st_date = now_ym - relativedelta(months=range_test_m)#テストデータに使用する区間を決める
    final_test_en_date = now_ym + relativedelta(months=range_final_m)#最終テストの終了日
    result_df_dummie=result_df_dummie[result_df_dummie['date']<final_test_en_date]#おおざっぱに切り分ける

    #クラスタリングに邪魔だから消したいけど、後々使うものはいったんよけておく
    result=result_df_dummie['result_com'].values.copy()#
    money=result_df_dummie['money'].values.copy()#
    years=result_df_dummie['year'].values.copy()#
    dates=result_df_dummie['date'].values.copy()#
    
    #安全なところに移したら削除する
    result_df_dummie=result_df_dummie.drop('result_com',axis=1)
    result_df_dummie=result_df_dummie.drop('money',axis=1)
    result_df_dummie=result_df_dummie.drop('year',axis=1)
    
    #いらない情報を削除したらデータの切り分け
    clustar_final_test_df=result_df_dummie[(result_df_dummie['date']>=now_ym) & (result_df_dummie['date']<final_test_en_date) ].copy()#今の日付以降を最終チェックデータ(予測のターゲット)に。
    clustar_test_df = result_df_dummie[(result_df_dummie['date']<now_ym) & ((result_df_dummie['date']>=test_st_date) )].copy()#今日に日より前の，指定した区間でのテストデータ
    clustar_train_df =  result_df_dummie[(result_df_dummie['date']<test_st_date)].copy()#そのほかを学習データに
    #日付の情報だけ切り分けに使ったからここで消す。
    clustar_final_test_df=clustar_final_test_df.drop('date',axis=1).copy()
    clustar_test_df=clustar_test_df.drop('date',axis=1).copy()
    clustar_train_df=clustar_train_df.drop('date',axis=1).copy()
    #esult_df_dummie=result_df_dummie.drop('date',axis=1)
    target_num_cluster=[3,5,7,9]#分けるクラスタ数によってモデルの名前を変える
    for num_cluster in target_num_cluster:
        Km = KMeans(random_state=7,n_clusters=num_cluster).fit(clustar_train_df)#rondom_stateはラッキーセブン
        #final_test_pred =Km.predict(clustar_final_test_df)#rondom_stateはラッキーセブン
        final_test_pred =Km.predict(clustar_final_test_df)#rondom_stateはラッキーセブン
        test_pred =Km.predict(clustar_test_df)#rondom_stateはラッキーセブン
        train_pred = Km.predict(clustar_train_df)#rondom_stateはラッキーセブン
        #Km=========================実査に使うときはこれのモデルを会場ごとに保存して使用。
        #clustar_final_test_df['num={}'.format(num_cluster)]=final_test_pred
        clustar_final_test_df['num={}'.format(num_cluster)]=final_test_pred
        clustar_test_df['num={}'.format(num_cluster)]=test_pred
        clustar_train_df['num={}'.format(num_cluster)]=train_pred

    #結合して元の形に戻す。
    #clustar_df=pd.concat([clustar_train_df, clustar_test_df,clustar_final_test_df]).copy()
#     clustar_final_test_df['check']='final'#確認用
#     clustar_test_df['check']='test'#確認用
#     clustar_train_df['check']='train'#確認用
    clustar_df=pd.concat([clustar_train_df, clustar_test_df,clustar_final_test_df]).copy()
    clustar_df['year']=years
    clustar_df['date']=dates
    clustar_df['money']=money
    clustar_df['result_com']=result
    model_df=clustar_df.copy()
    return model_df

def data_making_clustar_section(df,now_ym,range_test_m):#モデル関連に使用するdfの作成関数(クラスタリングあり、モータ番号、艇番号なし)（加工関数）final(実運用バックテスト区間)無し版
    #V4系列で使用する加工関数，年と月の情報を使って直近のデータを使って性能検証を行う（区間を使ってデータを作っている）
    #validデータを作成するバージョンなので実装する際はこれをそのまま使わず，final_test部分の処理は消してくだちい
    #yearが使わないと思うけど一応残してあるから邪魔だと思ったら消して下さい
    result_df=df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22}).copy()#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02}).copy()#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02}).copy()

    #result_base_df=result_df.copy()
    #result_base_df=get_event_info(result_base_df)#開催の情報について付与する関数(年月日に加えて，何日間の開催かどうかも教えてくれる)

    #ダミー変数化
    result_df_dummie=result_df.copy()
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1).copy()

    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr.copy()
        male_dummie_df=pd.get_dummies(result_df_dummie[col]).copy()#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val.copy()
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    cols=list(result_df_dummie.columns)



    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す

    #boat、moterの情報は使わない、
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()
    for col in boat_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    #クラスタリング
    #分けてみるクラスタの数は[3,5,7,9]の4個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    #学習データのdateを年に変換
    result_df_dummie['date']=pd.to_datetime(result_df_dummie['date'])#日付が文字列なのでdateを日付型に変換
    result_df_dummie['year']=result_df_dummie['date'].dt.year
    
    #==========================================================================
    #result_df_dummie=result_df_dummie[result_df_dummie['year']!=2020].copy()#2020のデータを完全に切り離す。
    #==========================================================================
    #クラアスタリング用の学習、予測用のデータの切り分け
    #年，月，日とかの取得
    #now_ym:開始の月
    test_st_date = now_ym - relativedelta(months=range_test_m)#テストデータに使用する区間を決める
    result_df_dummie=result_df_dummie[result_df_dummie['date']<now_ym]#おおざっぱに切り分ける
    #final_test_en_date = now_ym + relativedelta(months=range_final_m)#最終テストの終了日
    #クラスタリングに邪魔だから消したいけど、後々使うものはいったんよけておく
    result=result_df_dummie['result_com'].values.copy()#
    money=result_df_dummie['money'].values.copy()#
    years=result_df_dummie['year'].values.copy()#
    dates=result_df_dummie['date'].values.copy()#
    
    #安全なところに移したら削除する
    result_df_dummie=result_df_dummie.drop('result_com',axis=1)
    result_df_dummie=result_df_dummie.drop('money',axis=1)
    result_df_dummie=result_df_dummie.drop('year',axis=1)
    
    #切り分ける
    #clustar_final_test_df=result_df_dummie[(result_df_dummie['date']>=now_ym) & (result_df_dummie['date']<final_test_en_date) ].copy()#今の日付以降を最終チェックデータ(予測のターゲット)に。
    clustar_test_df = result_df_dummie[(result_df_dummie['date']<now_ym) & ((result_df_dummie['date']>=test_st_date) )].copy()#今日より前の，指定した区間でのテストデータ
    clustar_train_df =  result_df_dummie[(result_df_dummie['date']<test_st_date)].copy()#そのほかを学習データに
    #日付の情報だけ切り分けに使ったからここで消す。
    #clustar_final_test_df=clustar_final_test_df.drop('date',axis=1).copy()
    clustar_test_df=clustar_test_df.drop('date',axis=1).copy()
    clustar_train_df=clustar_train_df.drop('date',axis=1).copy()
    #result_df_dummie=result_df_dummie.drop('date',axis=1)

    target_num_cluster=[3,5,7,9]#分けるクラスタ数によってモデルの名前を変える
    for num_cluster in target_num_cluster:
        Km = KMeans(random_state=7,n_clusters=num_cluster).fit(clustar_train_df)#rondom_stateはラッキーセブン
        #final_test_pred =Km.predict(clustar_final_test_df)#rondom_stateはラッキーセブン
        test_pred =Km.predict(clustar_test_df)#rondom_stateはラッキーセブン
        train_pred = Km.predict(clustar_train_df)#rondom_stateはラッキーセブン
        #Km=========================実査に使うときはこれのモデルを会場ごとに保存して使用。
        #clustar_final_test_df['num={}'.format(num_cluster)]=final_test_pred
        #clustar_final_test_df['num={}'.format(num_cluster)]=final_test_pred
        clustar_test_df['num={}'.format(num_cluster)]=test_pred
        clustar_train_df['num={}'.format(num_cluster)]=train_pred

    #結合して元の形に戻す。
    #clustar_df=pd.concat([clustar_train_df, clustar_test_df,clustar_final_test_df]).copy()
#     clustar_final_test_df['check']='final'#確認用
#     clustar_test_df['check']='test'#確認用
#     clustar_train_df['check']='train'#確認用
    #clustar_df=pd.concat([clustar_train_df, clustar_test_df,clustar_final_test_df]).copy()
    clustar_df=pd.concat([clustar_train_df, clustar_test_df]).copy()
    clustar_df['year']=years
    clustar_df['date']=dates
    clustar_df['money']=money
    clustar_df['result_com']=result
    model_df=clustar_df.copy()
    return model_df

def data_making_clustar(df,years):#モデル関連に使用するdfの作成関数(クラスタリングあり、モータ番号、艇番号なし)
    result_df=df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22}).copy()#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02}).copy()#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02}).copy()

    year1=years[0]
    year2=years[1]

    #ダミー変数化
    result_df_dummie=result_df.copy()
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1).copy()

    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr.copy()
        male_dummie_df=pd.get_dummies(result_df_dummie[col]).copy()#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val.copy()
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    cols=list(result_df_dummie.columns)



    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す

    #boat、moterの情報は使わない、
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()
    for col in boat_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    #クラスタリング
    #分けてみるクラスタの数は[3,5,7,9]の4個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    #学習データのdateを年に変換
    result_df_dummie['date']=pd.to_datetime(result_df_dummie['date'])#日付が文字列なのでdateを日付型に変換
    result_df_dummie['year']=result_df_dummie['date'].dt.year

    #==========================================================================
    #result_df_dummie=result_df_dummie[result_df_dummie['year']!=2020].copy()#2020のデータを完全に切り離す。
    #==========================================================================

    #クラスタリングに邪魔だから消したいけど、後々使うものはいったんよけておく
    result=result_df_dummie['result_com'].values.copy()#
    money=result_df_dummie['money'].values.copy()#
    years=result_df_dummie['year'].values.copy()#

    #安全なところに移したら削除する
    result_df_dummie=result_df_dummie.drop('result_com',axis=1)
    result_df_dummie=result_df_dummie.drop('money',axis=1)
    result_df_dummie=result_df_dummie.drop('date',axis=1)
    #クラアスタリング用の学習、予測用のデータの切り分け
    #clustar_final_test_df=result_df_dummie[(result_df_dummie['year']==year3)].copy()#2020のデータを最終チェックデータ(予測のターゲット)に。
    clustar_test_df = result_df_dummie[(result_df_dummie['year']==year1) | ((result_df_dummie['year']==year2) )].copy()#2019,2010のデータを検証用データに。
    clustar_train_df =  result_df_dummie[(result_df_dummie['year']!=year1) & (result_df_dummie['year']!=year2)].copy()#そのほかを学習データに

    #年の情報だけ切り分けに使ったからここで消す。
    #clustar_final_test_df=clustar_final_test_df.drop('year',axis=1).copy()
    clustar_test_df=clustar_test_df.drop('year',axis=1).copy()
    clustar_train_df=clustar_train_df.drop('year',axis=1).copy()

    target_num_cluster=[3,5,7,9]#分けるクラスタ数によってモデルの名前を変える
    for num_cluster in target_num_cluster:
        Km = KMeans(random_state=7,n_clusters=num_cluster).fit(clustar_train_df)#rondom_stateはラッキーセブン
        #final_test_pred =Km.predict(clustar_final_test_df)#rondom_stateはラッキーセブン
        test_pred =Km.predict(clustar_test_df)#rondom_stateはラッキーセブン
        train_pred = Km.predict(clustar_train_df)#rondom_stateはラッキーセブン
        #Km=========================実査に使うときはこれのモデルを会場ごとに保存して使用。
        #clustar_final_test_df['num={}'.format(num_cluster)]=final_test_pred
        clustar_test_df['num={}'.format(num_cluster)]=test_pred
        clustar_train_df['num={}'.format(num_cluster)]=train_pred

    #結合して元の形に戻す。
    #clustar_df=pd.concat([clustar_train_df, clustar_test_df,clustar_final_test_df]).copy()
    clustar_df=pd.concat([clustar_train_df, clustar_test_df]).copy()
    clustar_df['year']=years
    clustar_df['money']=money
    clustar_df['result_com']=result
    model_df=clustar_df.copy()
    return model_df

def data_making_mo_bo(df):#クラスタリングなし、ボート、艇番号あり

    result_df=df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22})#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02})#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02})
    #ダミー変数化
    result_df_dummie=result_df
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1)

    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        male_dummie_df=pd.get_dummies(result_df_dummie[col])#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    cols=list(result_df_dummie.columns)
    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す
    #boat もmoterも番号は1~99とする
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        for number in numbers:
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        moter_dummie_df=pd.get_dummies(result_df_dummie[col])#モータ番号をダミー化
        for column, val in moter_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    #boat番号をダミー化
    for col in boat_cols:
        for number in numbers:
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        boat_dummie_df=pd.get_dummies(result_df_dummie[col])#boat番号をダミー化
        for column, val in boat_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    #クラスタリング
    #分けてみるクラスタの数は[8,10]の2個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    target_num_cluster=[8,10]
    #test_clustaring_df=train_has_PCA_df
    clustar_target_df=result_df_dummie
    clustaring_df=clustar_target_df
    """
    for num_cluster in target_num_cluster:
        pred = KMeans(random_state=0,n_clusters=num_cluster).fit_predict(clustar_target_df)
        clustaring_df['num={}'.format(num_cluster)]=pred
    """
    model_df=clustaring_df
    model_df['date']=pd.to_datetime(model_df['date'])#日付が文字列なのでdateを日付型に変換
    model_df['year']=model_df['date'].dt.year
    return model_df

def data_making_none_clustar(df):#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
    result_df=df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22}).copy()#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22}).copy()
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02}).copy()#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02}).copy()
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02}).copy()

    #ダミー変数化
    result_df_dummie=result_df.copy()
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1).copy()

    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr.copy()
        male_dummie_df=pd.get_dummies(result_df_dummie[col]).copy()#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val.copy()
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    cols=list(result_df_dummie.columns)



    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す

    #boat、moterの情報は使わない、
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()
    for col in boat_cols:
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1).copy()

    #クラスタリング
    #分けてみるクラスタの数は[3,5,7,9]の4個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    #学習データのdateを年に変換
    result_df_dummie['date']=pd.to_datetime(result_df_dummie['date'])#日付が文字列なのでdateを日付型に変換
    result_df_dummie['year']=result_df_dummie['date'].dt.year
    return result_df_dummie
def model_score_rondom_forest(version,place_name,result_df):#学習データと場所名を渡せば探索を初めて、指定のディレクトリにスコアをまとめたcsvを出力する。
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])#スコアを格納するdf

    #学習データの切り分け
    test_df = result_df[(result_df['year']==2019) | ((result_df['year']==2020) )]#2019,2020のデータを検証用データに。
    train_df =  result_df[(result_df['year']!=2019) & ((result_df['year']!=2020) )]#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_df=test_df.drop(['year'],axis=1)
    train_df=train_df.drop(['year'],axis=1)

    train_money=pd.Series(train_df['money'])
    test_money=pd.Series(test_df['money'])

    #x,yへの切り分け
    #出現数の分布
    result_com_s=test_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    gain_mean=test_df.groupby('result_com')['money'].mean()
    gain_mean=gain_mean.sort_index()

    gain_median=test_df.groupby('result_com')['money'].median()
    gain_median=gain_median.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                                'result_com_num':result_com_s.values,
                                'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                                'gain_mean':gain_mean.values,
                                'gain_median':gain_median.values,})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため




    for result_com_number in tqdm(result_com_df['result_com'].values):
        #print(result_com_number)
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================

        gain_th=10#利益率の閾値
        result_s=result_com_df[result_com_df['result_com']==result_com]
        buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
        num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_arr=[0]*len(result_train_df)
        i=0
        for result in result_train_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1
        result_train_df['result_com']=result_arr
        result_test_df=test_df.copy()
        result_arr=[0]*len(result_test_df)
        i=0
        for result in result_test_df['result_com']:
            if ((result==result_com)):
                result_arr[i]=1
            else:
                result_arr[i]=0
            i+=1

        result_test_df['result_com']=result_arr

        result_train_df['money']=train_money
        result_test_df['money']=test_money
        #学習データラベル変換終わり============================================

        for_arr=np.arange(1,73)
        #for_arr=np.arange(1,100,1)
        accuracy_arr=[0]*len(for_arr)
        target_per_arr=[0]*len(for_arr)
        pred_0=[0]*len(for_arr)
        gain_arr=[0]*len(for_arr)
        model_gain_arr=[0]*len(result_test_df)
        test_gain_arr=test_money.values
        #depths_arr=[4,5,6,7,8]
        depths_arr=[5,6,8]
        for depth in depths_arr:
            for sum_target_per in for_arr:

                index=sum_target_per-1
                #target_per=50+sum_target_per
                target_per=80+(sum_target_per)
                target_per_arr[index]=target_per

                #モデルの評価指標値を格納するseries======================
                model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'], dtype='float64')
                model_score_s['target_com']=result_com#目標としているresult_comラベル番号
                model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
                model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
                #======================
                #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
                # 一層目の判別機のtrainデータ　:terget_result_df
                target_df=result_train_df#ベースのデータフレームをコピー
                target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
                target_1_df=target_df[target_df['result_com']==1]
                len_1=len(target_1_df)
                target_0_df=target_df[target_df['result_com']==0]
                len_0=len(target_0_df)
                target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
                target_train_df=pd.concat([target_1_df, target_0_df])
                #学習＆予測ぱーと========================================================================
                #==========================================================================================================================================
                #データの切り分け
                target_x_train=target_train_df.drop('money',axis=1)
                target_x_train=target_x_train.drop('result_com',axis=1)
                target_x_test=result_test_df.drop('money',axis=1)
                target_x_test=target_x_test.drop('result_com',axis=1)

                target_y_train=target_train_df['result_com']
                target_y_test=result_test_df['result_com']

                #テストデータ
                RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth,n_jobs=10)
                RF = RF.fit(target_x_train,target_y_train)
                # 未知データに対する予測値
                predict_y_test = RF.predict(target_x_test)
                #==========================================================================================================================================
                #==========================================================================================================================================
                #==========================================================================================================================================
                #==========================================================================================================================================

                #[1]の正答率を見る
                pred_test_df=pd.DataFrame({'pred':predict_y_test
                                          , 'test':target_y_test})
                num_1=len(pred_test_df[pred_test_df['test']==1])
                count=0
                #追加　配当金の情報も考慮する。
                gain_index=0
                model_gain_arr=[0]*len(result_test_df)
                for _, s in pred_test_df.iterrows():
                    if ((s['pred']==1) and (s['test']==1)):
                        count+=1#的中回数
                        model_gain_arr[gain_index]=test_gain_arr[gain_index]
                    gain_index+=1
                #print('test accyracy: {}'.format((count/num_1)*100))
                gain_arr[index]=sum(model_gain_arr)
                accuracy_arr[index]=(count/num_1)*100
                try:
                    pred_0[index]=pred_test_df['pred'].value_counts()[0]
                except:
                    pred_0[index]=0
                #scoreのseriesに情報書き込み==================
                model_score_s['総収益']=sum(model_gain_arr)
                model_score_s['投資金額']=100*sum(predict_y_test)
                model_score_s['出現数']=sum(target_y_test)
                model_score_s['購買予測数']=sum(predict_y_test)
                model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['購買的中率']=(count/sum(predict_y_test))*100
                model_score_s['的中数']=count
                model_score_df=model_score_df.append(model_score_s,ignore_index=True)


    #モデルの「スコアを保存
    #model_score_df.to_csv('{}_model_score.csv'.format(place), encoding='utf_8_sig')
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/{place_name}_model_score_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先#使用するデータの読み込み
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

## 実行する関数(今回作成した関数)

In [6]:
def model_score_pc_et_year(version,years,place_name,result_df):#pycaretを使ってモデリング(et),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,10,25,50,100,200,500]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('rf')#モデルの生成（Extra Trees Regressor）
            tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
            predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_pc_et_tuneless_year(version,years,place_name,result_df):#pycaretを使ってモデリング(et),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,10,25,50,100,200,500]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('rf')#モデルの生成（Extra Trees Regressor）
            #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(model, data=target_x_year1_test)
            predict_year2_df = predict_model(model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/tuneless_{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_pc_nb_tuneless_year(version,years,place_name,result_df):#pycaretを使ってモデリング(Naive beise),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[100,200,500,700,1000,3000,5000]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('nb')#モデルの生成（Extra Trees Regressor）
            #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(model, data=target_x_year1_test)
            predict_year2_df = predict_model(model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/tuneless_nb_{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_pc_nb_year(version,years,place_name,result_df):#pycaretを使ってモデリング(Naive beise),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[100,200,500,700,1000,3000,5000]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('nb')#モデルの生成（Extra Trees Regressor）
            tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
            predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/{place_name}_nb_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_pc_knn_year(version,years,place_name,result_df):#pycaretを使ってモデリング(Naive beise),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,10,25,50,100,200,500]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('knn')#モデルの生成（Extra Trees Regressor）
            tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
            predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/{place_name}_knn_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_pc_knn_tuneless_year(version,years,place_name,result_df):#pycaretを使ってモデリング(Naive beise),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,10,25,50,100,200,500]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('knn')#モデルの生成（Extra Trees Regressor）
            #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(model, data=target_x_year1_test)
            predict_year2_df = predict_model(model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/tuneless_knn_{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None


## 実行(年単位バージョン)

# クラスタリングなし，ボート，モータ番号あり

In [33]:
def model_score_number_pc_et_year(version,years,place_name,result_df):#pycaretを使ってモデリング(et),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,10,25,50,100,200,500]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('rf')#モデルの生成（Extra Trees Regressor）
            tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
            predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_number_pc_et_tuneless_year(version,years,place_name,result_df):#pycaretを使ってモデリング(et),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,10,25,50,100,200,500]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('rf')#モデルの生成（Extra Trees Regressor）
            #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(model, data=target_x_year1_test)
            predict_year2_df = predict_model(model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_tuneless_{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_number_pc_nb_tuneless_year(version,years,place_name,result_df):#pycaretを使ってモデリング(Naive beise),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[100,200,500,700,1000,3000,5000]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('nb')#モデルの生成（Extra Trees Regressor）
            #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(model, data=target_x_year1_test)
            predict_year2_df = predict_model(model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_tuneless_nb_{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_number_pc_nb_year(version,years,place_name,result_df):#pycaretを使ってモデリング(Naive beise),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[100,200,500,700,1000,3000,5000]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('nb')#モデルの生成（Extra Trees Regressor）
            tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
            predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_{place_name}_nb_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_number_pc_knn_year(version,years,place_name,result_df):#pycaretを使ってモデリング(Naive beise),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,10,25,50,100,200,500]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('knn')#モデルの生成（Extra Trees Regressor）
            tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
            predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_{place_name}_knn_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_number_pc_knn_tuneless_year(version,years,place_name,result_df):#pycaretを使ってモデリング(Naive beise),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,10,25,50,100,200,500]
        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            #exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
            exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10,silent=True)#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('knn')#モデルの生成（Extra Trees Regressor）
            #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(model, data=target_x_year1_test)
            predict_year2_df = predict_model(model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_tuneless_knn_{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None


In [34]:
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_1'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)

years=[2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
result_df=result_df.drop('date',axis=1)
model_score_number_pc_et_year(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8889,0.8261,0.2500,1.0000,0.4000,0.3622,0.4703
1,0.7963,0.7092,0.3750,0.3333,0.3529,0.2326,0.2331
2,0.8704,0.7853,0.2500,0.6667,0.3636,0.3077,0.3540
3,0.9074,0.8505,0.3750,1.0000,0.5455,0.5055,0.5816
4,0.8148,0.8342,0.2500,0.3333,0.2857,0.1818,0.1843
5,0.8148,0.6359,0.2500,0.3333,0.2857,0.1818,0.1843
6,0.8519,0.8315,0.2500,0.5000,0.3333,0.2603,0.2801
7,0.8889,0.7720,0.4286,0.6000,0.5000,0.4394,0.4473
8,0.7925,0.5870,0.1429,0.1667,0.1538,0.0364,0.0365
9,0.8868,0.8975,0.2857,0.6667,0.4000,0.3484,0.3868


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [8:15:19<00:00, 1061.43s/it]

DONE


In [35]:
result_df

,result_com,money,racer_1_rank,racer_1_age,racer_1_doub,racer_1_ave_st,racer_2_rank,racer_2_age,racer_2_doub,racer_2_ave_st,...,racer_6_bo_91,racer_6_bo_92,racer_6_bo_93,racer_6_bo_94,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,year
0,6,1160.0,4,35.0,0.541,0.14,2,40.0,0.287,0.19,...,0,0,0,0,0,0,0,0,0,2012
1,3,2960.0,4,46.0,0.450,0.18,2,27.0,0.301,0.18,...,0,0,0,0,0,0,0,0,0,2012
2,44,21490.0,3,43.0,0.480,0.16,2,35.0,0.255,0.21,...,0,0,0,0,0,0,0,0,0,2012
3,62,5100.0,4,27.0,0.506,0.17,2,38.0,0.188,0.20,...,0,0,0,0,0,0,0,0,0,2012
4,25,1980.0,3,32.0,0.439,0.21,4,25.0,0.536,0.17,...,0,0,0,0,0,0,0,0,0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21148,3,1280.0,3,40.0,0.357,0.15,2,22.0,0.159,0.20,...,0,0,0,0,0,0,0,0,0,2021
21149,24,1880.0,2,44.0,0.337,0.17,4,35.0,0.473,0.16,...,0,0,0,0,0,0,0,0,0,2021
21150,9,4630.0,3,33.0,0.342,0.16,4,41.0,0.460,0.21,...,0,0,0,0,0,0,0,0,0,2021
21151,13,2370.0,3,40.0,0.398,0.15,2,34.0,0.292,0.18,...,0,0,0,0,0,0,0,0,0,2021


In [36]:
#ハイパーパラメータ探索を行わない
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_1'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
result_df=result_df.drop('date',axis=1)
model_score_number_pc_et_tuneless_year(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8519,0.6685,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.8519,0.6644,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.8519,0.6399,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.8519,0.7826,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.8519,0.7283,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.8519,0.5924,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.8519,0.8573,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.8704,0.6915,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.8679,0.6242,0.0000,0.0000,0.0000,0.0000,0.0000
9,0.8679,0.8261,0.0000,0.0000,0.0000,0.0000,0.0000


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [57:29<00:00, 123.18s/it]

DONE


In [37]:
#ナイーブベイズ版,ためしにチューニングなし
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_1'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
result_df=result_df.drop('date',axis=1)
model_score_number_pc_nb_tuneless_year(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9726,0.4962,0.0000,0.0000,0.0000,-0.0110,-0.0124
1,0.9751,0.4975,0.0000,0.0000,0.0000,-0.0080,-0.0101
2,0.9701,0.4949,0.0000,0.0000,0.0000,-0.0135,-0.0143
3,0.9725,0.4962,0.0000,0.0000,0.0000,-0.0110,-0.0124
4,0.9700,0.4949,0.0000,0.0000,0.0000,-0.0135,-0.0144
5,0.9750,0.4974,0.0000,0.0000,0.0000,-0.0081,-0.0101
6,0.9650,0.4923,0.0000,0.0000,0.0000,-0.0174,-0.0176
7,0.9775,0.4975,0.0000,0.0000,0.0000,-0.0078,-0.0095
8,0.9675,0.4924,0.0000,0.0000,0.0000,-0.0164,-0.0165
9,0.9800,0.4987,0.0000,0.0000,0.0000,-0.0044,-0.0067


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [1:00:09<00:00, 128.91s/it]

DONE


In [38]:

#ナイーブベイズ版
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_1'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
result_df=result_df.drop('date',axis=1)
model_score_number_pc_nb_year(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9776,0.7013,0.0000,0.0000,0.0000,-0.0045,-0.0071
1,0.9825,0.6517,0.1250,1.0000,0.2222,0.2188,0.3504
2,0.9800,0.7010,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.9750,0.7860,0.0000,0.0000,0.0000,-0.0081,-0.0101
4,0.9750,0.7733,0.0000,0.0000,0.0000,-0.0081,-0.0101
5,0.9750,0.6798,0.0000,0.0000,0.0000,-0.0081,-0.0101
6,0.9775,0.8214,0.0000,0.0000,0.0000,-0.0045,-0.0072
7,0.9825,0.7990,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.9775,0.7303,0.0000,0.0000,0.0000,-0.0078,-0.0095
9,0.9825,0.8499,0.1429,0.5000,0.2222,0.2161,0.2608


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [1:33:16<00:00, 199.86s/it]

DONE


In [39]:

#knnn版
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_1'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
result_df=result_df.drop('date',axis=1)
model_score_number_pc_knn_year(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7963,0.4062,0.0000,0.0000,0.0000,-0.0879,-0.1011
1,0.8148,0.4755,0.1250,0.2500,0.1667,0.0753,0.0811
2,0.8148,0.4878,0.1250,0.2500,0.1667,0.0753,0.0811
3,0.8333,0.6495,0.1250,0.3333,0.1818,0.1099,0.1264
4,0.8333,0.5367,0.1250,0.3333,0.1818,0.1099,0.1264
5,0.8333,0.5774,0.2500,0.4000,0.3077,0.2186,0.2265
6,0.7963,0.4796,0.0000,0.0000,0.0000,-0.0879,-0.1011
7,0.8148,0.4939,0.1429,0.2000,0.1667,0.0657,0.0669
8,0.7925,0.3913,0.0000,0.0000,0.0000,-0.1063,-0.1115
9,0.8491,0.5466,0.1429,0.3333,0.2000,0.1311,0.1456


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [4:45:07<00:00, 610.97s/it]

DONE


In [40]:

#knn版
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_1'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
result_df=result_df.drop('date',axis=1)
model_score_number_pc_knn_tuneless_year(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8148,0.2717,0.0000,0.0000,0.0000,-0.0630,-0.0818
1,0.8333,0.4130,0.0000,0.0000,0.0000,-0.0340,-0.0573
2,0.8148,0.3927,0.0000,0.0000,0.0000,-0.0630,-0.0818
3,0.8333,0.4647,0.0000,0.0000,0.0000,-0.0340,-0.0573
4,0.8148,0.5367,0.0000,0.0000,0.0000,-0.0630,-0.0818
5,0.7963,0.4973,0.0000,0.0000,0.0000,-0.0879,-0.1011
6,0.7963,0.5408,0.0000,0.0000,0.0000,-0.0879,-0.1011
7,0.8704,0.5729,0.1429,0.5000,0.2222,0.1747,0.2162
8,0.8302,0.6211,0.0000,0.0000,0.0000,-0.0624,-0.0773
9,0.8491,0.5342,0.0000,0.0000,0.0000,-0.0341,-0.0541


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [1:06:30<00:00, 142.53s/it]

DONE


# 不均衡データへのアプローチを試すZ

In [7]:
def model_score_number_pc_rf_year_imb_3years(version,years,place_name,result_df):#pycaretを使ってモデリング(rf),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    #model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])#スコアを格納するdf
    model_score_df=pd.DataFrame(columns=['target_com','target_per','threshold','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    result_df=result_df.drop(['date'],axis=1)#このバージョンでは日付のデータは使わない
    
    year1=years[0]
    year2=years[1]
    year3=years[2]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2018のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2018のデータ
    test_year3_df= result_df[(result_df['year']==year3)].copy()#2018のデータ
    
    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)& (result_df['year']!=year3) ].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    #final_test_df=final_test_df.drop(['year'],axis=1).copy()
    #test_df=test_df.drop(['year'],axis=1).copy()
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()
    test_year3_df=test_year3_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    #final_test_money=pd.Series(final_test_df['money']).copy()
    #test_money=pd.Series(test_df['money']).copy()
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    test_year3_money=pd.Series(test_year3_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    #result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため
    result_com_df=result_com_df.iloc[0:70]#探索的に探しはするが　，時間の都合上一部のみ探索を行う
    
    for result_com_number in tqdm(result_com_df['result_com'].values):
        try:
            result_com=result_com_number

            #result_comごとの閾値の決定========================================================================
            result_s=result_com_df[result_com_df['result_com']==result_com]
            #===============================================================================
            #学習データのラベル変換==========================================================
            result_train_df=train_df.copy()
            result_train_df=trans_result_com(result_com,result_train_df)

            result_test_year1_df=test_year1_df.copy()
            result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
            result_test_year2_df=test_year2_df.copy()
            result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
            result_test_year3_df=test_year3_df.copy()
            result_test_year3_df=trans_result_com(result_com,result_test_year3_df)#対象のラベルを１、それ以外を０に変換する関数

            result_train_df['money']=train_money
            result_test_year1_df['money']=test_year1_money
            result_test_year2_df['money']=test_year2_money
            result_test_year3_df['money']=test_year3_money
            #学習データラベル変換終わり============================================

            target_per_arr=[500,1000,2000,3000,5000,10000]
            #target_per_arr=[20]
            for sum_target_per in target_per_arr:
                #index=sum_target_per-1
                target_per=100+sum_target_per#学習データを増やす
                #target_per_arr[index]=target_per
                #======================
                #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
                # 一層目の判別機のtrainデータ　:terget_result_df
                target_df=result_train_df.copy()#ベースのデータフレームをコピー
                target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
                target_1_df=target_df[target_df['result_com']==1].copy()
                len_1=len(target_1_df)

                target_0_df=target_df[target_df['result_com']==0].copy()
                len_0=len(target_0_df)
                target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
                target_train_df=pd.concat([target_1_df, target_0_df]).copy()
                #学習＆予測ぱーと========================================================================
                #==========================================================================================================================================
                #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
                target_com_train=target_train_df.drop('money',axis=1).copy()
        #             target_x_train=target_x_train.drop('result_com',axis=1)
        #             target_y_train=target_train_df['result_com'].copy()

                target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
                target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)
                target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
                target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)
                target_x_year3_test=result_test_year3_df.drop('money',axis=1).copy()
                target_x_year3_test=target_x_year3_test.drop('result_com',axis=1)

                target_y_year1_test=result_test_year1_df['result_com'].copy()
                target_y_year2_test=result_test_year2_df['result_com'].copy()
                target_y_year3_test=result_test_year3_df['result_com'].copy()

                #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
                exp = setup(target_com_train,target='result_com',
                            use_gpu=True,
                            train_size=0.7,
                            data_split_shuffle=False,
                            n_jobs=10,
                            silent=True,
                            fix_imbalance=True,
                            session_id=7#rondom_stateみたいなもん，再現性を持たせる
                            #fix_imbalance_method=ADASYN(sampling_strategy='minority')
                           )#学習環境の作成

                #compare_models( errors="raise")
                model=create_model('rf')#モデルの生成（Extra Trees Regressor）
                #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
                tuned_model=tune_model(model,optimize ='F1',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う

                #predict_model(tuned_model)
                # 未知データに対する予測値
                predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
                predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
                predict_year3_df = predict_model(tuned_model, data=target_x_year3_test)

                #==========================================================================================================================================
                #[1]の正答率を見る
                pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year1_test})
                pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year2_test})
                pred_year3_test_df=pd.DataFrame({'pred':predict_year3_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year3_test})
                year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
                year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
                year3_trans_df=pred_year3_test_df.copy()#閾値での変換は行わない
                #/////収益計算の項
                year1_trans_df['money']=test_year1_money
                year1_trans_df['true_result']=test_year1_df['result_com']
                year2_trans_df['money']=test_year2_money
                year2_trans_df['true_result']=test_year2_df['result_com']
                year3_trans_df['money']=test_year3_money
                year3_trans_df['true_result']=test_year3_df['result_com']
                #配当金の情報も考慮する。
                #result_gain_base_df=calc_gain(trans_df)
                year1_result_gain_base_df=calc_gain(year1_trans_df)
                year2_result_gain_base_df=calc_gain(year2_trans_df)
                year3_result_gain_base_df=calc_gain(year3_trans_df)

                #scoreのseriesに情報書き込み==================
                model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')

                model_score_s['target_com']=result_com#目標としているresult_comラベル番号
                model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
                #model_score_s['threshold']=th

                result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df,year3_result_gain_base_df]
                year_labels=[1,2,3]
                #年のごとのスコア情報を横に展開していく
                for year_df,label in zip(result_gain_df_arr,year_labels):
                    if label !=3:
                        #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                        model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                        #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                        model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                        #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                        model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                        #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                        model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                        #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                        model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                        #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                        model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                        #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                        model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
                    else:
                        model_score_s['gain_year{year}'.format(year=label)]=(year_df["gain"].sum()/(100*year_df["pred"].sum()))*100
                model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
                #モデルの「スコアを保存
                dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_{place_name}_model_score_{V}_test_imb_rf.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
                model_score_df.to_csv(dir_path, encoding='utf_8_sig')#定期的に吐き出す方針に変更した
        except:
            print('Error')
    return None

def model_score_number_pc_gbc_year_imb_3years(version,years,place_name,result_df):#pycaretを使ってモデリング(et),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    #model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])#スコアを格納するdf
    model_score_df=pd.DataFrame(columns=['target_com','target_per','threshold','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    result_df=result_df.drop(['date'],axis=1)#このバージョンでは日付のデータは使わない
    
    year1=years[0]
    year2=years[1]
    year3=years[2]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2018のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2018のデータ
    test_year3_df= result_df[(result_df['year']==year3)].copy()#2018のデータ
    
    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)& (result_df['year']!=year3) ].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    #final_test_df=final_test_df.drop(['year'],axis=1).copy()
    #test_df=test_df.drop(['year'],axis=1).copy()
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()
    test_year3_df=test_year3_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    #final_test_money=pd.Series(final_test_df['money']).copy()
    #test_money=pd.Series(test_df['money']).copy()
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    test_year3_money=pd.Series(test_year3_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    #result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため
    result_com_df=result_com_df.iloc[0:30]#探索的に探しはするが　，時間の都合上一部のみ探索を行う(このアルゴリズムはそんなに動かしたことが無いため，さらに短く．)
    
    for result_com_number in tqdm(result_com_df['result_com'].values):
        try:
            result_com=result_com_number

            #result_comごとの閾値の決定========================================================================
            result_s=result_com_df[result_com_df['result_com']==result_com]
            #===============================================================================
            #学習データのラベル変換==========================================================
            result_train_df=train_df.copy()
            result_train_df=trans_result_com(result_com,result_train_df)

            result_test_year1_df=test_year1_df.copy()
            result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
            result_test_year2_df=test_year2_df.copy()
            result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
            result_test_year3_df=test_year3_df.copy()
            result_test_year3_df=trans_result_com(result_com,result_test_year3_df)#対象のラベルを１、それ以外を０に変換する関数

            result_train_df['money']=train_money
            result_test_year1_df['money']=test_year1_money
            result_test_year2_df['money']=test_year2_money
            result_test_year3_df['money']=test_year3_money
            #学習データラベル変換終わり============================================

            target_per_arr=[100,300,500,1000,5000,10000]

            #target_per_arr=[20]
            for sum_target_per in target_per_arr:
                #index=sum_target_per-1
                target_per=100+sum_target_per#学習データを増やす
                #target_per_arr[index]=target_per
                #======================
                #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
                # 一層目の判別機のtrainデータ　:terget_result_df
                target_df=result_train_df.copy()#ベースのデータフレームをコピー
                target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
                target_1_df=target_df[target_df['result_com']==1].copy()
                len_1=len(target_1_df)

                target_0_df=target_df[target_df['result_com']==0].copy()
                len_0=len(target_0_df)
                target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
                target_train_df=pd.concat([target_1_df, target_0_df]).copy()
                #学習＆予測ぱーと========================================================================
                #==========================================================================================================================================
                #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
                target_com_train=target_train_df.drop('money',axis=1).copy()
        #             target_x_train=target_x_train.drop('result_com',axis=1)
        #             target_y_train=target_train_df['result_com'].copy()

                target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
                target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)
                target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
                target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)
                target_x_year3_test=result_test_year3_df.drop('money',axis=1).copy()
                target_x_year3_test=target_x_year3_test.drop('result_com',axis=1)

                target_y_year1_test=result_test_year1_df['result_com'].copy()
                target_y_year2_test=result_test_year2_df['result_com'].copy()
                target_y_year3_test=result_test_year3_df['result_com'].copy()

                #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
                exp = setup(target_com_train,target='result_com',
                            use_gpu=True,
                            train_size=0.7,
                            data_split_shuffle=False,
                            n_jobs=10,
                            silent=True,
                            fix_imbalance=True,
                            session_id=7#rondom_stateみたいなもん，再現性を持たせる
                            #fix_imbalance_method=ADASYN(sampling_strategy='minority')
                           )#学習環境の作成

                #compare_models( errors="raise")
                model=create_model('gbc')#モデルの生成（Extra Trees Regressor）
                #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
                tuned_model=tune_model(model,optimize ='F1',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う

                #predict_model(tuned_model)
                # 未知データに対する予測値
                predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
                predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
                predict_year3_df = predict_model(tuned_model, data=target_x_year3_test)

                #==========================================================================================================================================
                #[1]の正答率を見る
                pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year1_test})
                pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year2_test})
                pred_year3_test_df=pd.DataFrame({'pred':predict_year3_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year3_test})
                year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
                year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
                year3_trans_df=pred_year3_test_df.copy()#閾値での変換は行わない
                #/////収益計算の項
                year1_trans_df['money']=test_year1_money
                year1_trans_df['true_result']=test_year1_df['result_com']
                year2_trans_df['money']=test_year2_money
                year2_trans_df['true_result']=test_year2_df['result_com']
                year3_trans_df['money']=test_year3_money
                year3_trans_df['true_result']=test_year3_df['result_com']
                #配当金の情報も考慮する。
                #result_gain_base_df=calc_gain(trans_df)
                year1_result_gain_base_df=calc_gain(year1_trans_df)
                year2_result_gain_base_df=calc_gain(year2_trans_df)
                year3_result_gain_base_df=calc_gain(year3_trans_df)

                #scoreのseriesに情報書き込み==================
                model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')

                model_score_s['target_com']=result_com#目標としているresult_comラベル番号
                model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
                #model_score_s['threshold']=th

                result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df,year3_result_gain_base_df]
                year_labels=[1,2,3]
                #年のごとのスコア情報を横に展開していく
                for year_df,label in zip(result_gain_df_arr,year_labels):
                    if label !=3:
                        #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                        model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                        #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                        model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                        #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                        model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                        #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                        model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                        #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                        model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                        #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                        model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                        #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                        model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
                    else:
                        model_score_s['gain_year{year}'.format(year=label)]=(year_df["gain"].sum()/(100*year_df["pred"].sum()))*100
                model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
                #モデルの「スコアを保存
                dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_{place_name}_model_score_{V}_test_imb_gbc.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
                model_score_df.to_csv(dir_path, encoding='utf_8_sig')#定期的に吐き出す方針に変更した
        except:
            print('Error')
    return None

def model_score_number_pc_ada_year_imb_3years(version,years,place_name,result_df):#pycaretを使ってモデリング(ada,年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    #model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])#スコアを格納するdf
    model_score_df=pd.DataFrame(columns=['target_com','target_per','threshold','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    result_df=result_df.drop(['date'],axis=1)#このバージョンでは日付のデータは使わない
    
    year1=years[0]
    year2=years[1]
    year3=years[2]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2018のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2018のデータ
    test_year3_df= result_df[(result_df['year']==year3)].copy()#2018のデータ
    
    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)& (result_df['year']!=year3) ].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    #final_test_df=final_test_df.drop(['year'],axis=1).copy()
    #test_df=test_df.drop(['year'],axis=1).copy()
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()
    test_year3_df=test_year3_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    #final_test_money=pd.Series(final_test_df['money']).copy()
    #test_money=pd.Series(test_df['money']).copy()
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    test_year3_money=pd.Series(test_year3_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    #result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため
    result_com_df=result_com_df.iloc[0:30]#探索的に探しはするが　，時間の都合上一部のみ探索を行う(このアルゴリズムはそんなに動かしたことが無いため，さらに短く．)
    
    for result_com_number in tqdm(result_com_df['result_com'].values):
        try:
            result_com=result_com_number

            #result_comごとの閾値の決定========================================================================
            result_s=result_com_df[result_com_df['result_com']==result_com]
            #===============================================================================
            #学習データのラベル変換==========================================================
            result_train_df=train_df.copy()
            result_train_df=trans_result_com(result_com,result_train_df)

            result_test_year1_df=test_year1_df.copy()
            result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
            result_test_year2_df=test_year2_df.copy()
            result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
            result_test_year3_df=test_year3_df.copy()
            result_test_year3_df=trans_result_com(result_com,result_test_year3_df)#対象のラベルを１、それ以外を０に変換する関数

            result_train_df['money']=train_money
            result_test_year1_df['money']=test_year1_money
            result_test_year2_df['money']=test_year2_money
            result_test_year3_df['money']=test_year3_money
            #学習データラベル変換終わり============================================

            target_per_arr=[100,300,500,1000,5000,10000]

            #target_per_arr=[20]
            for sum_target_per in target_per_arr:
                #index=sum_target_per-1
                target_per=100+sum_target_per#学習データを増やす
                #target_per_arr[index]=target_per
                #======================
                #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
                # 一層目の判別機のtrainデータ　:terget_result_df
                target_df=result_train_df.copy()#ベースのデータフレームをコピー
                target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
                target_1_df=target_df[target_df['result_com']==1].copy()
                len_1=len(target_1_df)

                target_0_df=target_df[target_df['result_com']==0].copy()
                len_0=len(target_0_df)
                target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
                target_train_df=pd.concat([target_1_df, target_0_df]).copy()
                #学習＆予測ぱーと========================================================================
                #==========================================================================================================================================
                #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
                target_com_train=target_train_df.drop('money',axis=1).copy()
        #             target_x_train=target_x_train.drop('result_com',axis=1)
        #             target_y_train=target_train_df['result_com'].copy()

                target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
                target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)
                target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
                target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)
                target_x_year3_test=result_test_year3_df.drop('money',axis=1).copy()
                target_x_year3_test=target_x_year3_test.drop('result_com',axis=1)

                target_y_year1_test=result_test_year1_df['result_com'].copy()
                target_y_year2_test=result_test_year2_df['result_com'].copy()
                target_y_year3_test=result_test_year3_df['result_com'].copy()

                #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
                exp = setup(target_com_train,target='result_com',
                            use_gpu=True,
                            train_size=0.7,
                            data_split_shuffle=False,
                            n_jobs=10,
                            silent=True,
                            fix_imbalance=True,
                            session_id=7#rondom_stateみたいなもん，再現性を持たせる
                            #fix_imbalance_method=ADASYN(sampling_strategy='minority')
                           )#学習環境の作成

                #compare_models( errors="raise")
                model=create_model('ada')#モデルの生成（Extra Trees Regressor）
                #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
                tuned_model=tune_model(model,optimize ='F1',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う

                #predict_model(tuned_model)
                # 未知データに対する予測値
                predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
                predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
                predict_year3_df = predict_model(tuned_model, data=target_x_year3_test)

                #==========================================================================================================================================
                #[1]の正答率を見る
                pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year1_test})
                pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year2_test})
                pred_year3_test_df=pd.DataFrame({'pred':predict_year3_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year3_test})
                year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
                year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
                year3_trans_df=pred_year3_test_df.copy()#閾値での変換は行わない
                #/////収益計算の項
                year1_trans_df['money']=test_year1_money
                year1_trans_df['true_result']=test_year1_df['result_com']
                year2_trans_df['money']=test_year2_money
                year2_trans_df['true_result']=test_year2_df['result_com']
                year3_trans_df['money']=test_year3_money
                year3_trans_df['true_result']=test_year3_df['result_com']
                #配当金の情報も考慮する。
                #result_gain_base_df=calc_gain(trans_df)
                year1_result_gain_base_df=calc_gain(year1_trans_df)
                year2_result_gain_base_df=calc_gain(year2_trans_df)
                year3_result_gain_base_df=calc_gain(year3_trans_df)

                #scoreのseriesに情報書き込み==================
                model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')

                model_score_s['target_com']=result_com#目標としているresult_comラベル番号
                model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
                #model_score_s['threshold']=th

                result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df,year3_result_gain_base_df]
                year_labels=[1,2,3]
                #年のごとのスコア情報を横に展開していく
                for year_df,label in zip(result_gain_df_arr,year_labels):
                    if label !=3:
                        #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                        model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                        #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                        model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                        #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                        model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                        #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                        model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                        #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                        model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                        #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                        model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                        #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                        model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
                    else:
                        model_score_s['gain_year{year}'.format(year=label)]=(year_df["gain"].sum()/(100*year_df["pred"].sum()))*100
                model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
                #モデルの「スコアを保存
                dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_{place_name}_model_score_{V}_test_imb_ada.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
                model_score_df.to_csv(dir_path, encoding='utf_8_sig')#定期的に吐き出す方針に変更した
        except:
            print('Error')
    return None

def model_score_number_pc_rf_year_imb_3years_5950(version,years,place_name,result_df):#pycaretを使ってモデリング(rf),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    #model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','threshold','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])#スコアを格納するdf
    model_score_df=pd.DataFrame(columns=['target_com','target_per','threshold','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    result_df=result_df.drop(['date'],axis=1)#このバージョンでは日付のデータは使わない
    
    year1=years[0]
    year2=years[1]
    year3=years[2]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2018のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2018のデータ
    test_year3_df= result_df[(result_df['year']==year3)].copy()#2018のデータ
    
    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)& (result_df['year']!=year3) ].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    #final_test_df=final_test_df.drop(['year'],axis=1).copy()
    #test_df=test_df.drop(['year'],axis=1).copy()
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()
    test_year3_df=test_year3_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    #final_test_money=pd.Series(final_test_df['money']).copy()
    #test_money=pd.Series(test_df['money']).copy()
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    test_year3_money=pd.Series(test_year3_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    #result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため
    result_com_df=result_com_df.iloc[0:70]#探索的に探しはするが　，時間の都合上一部のみ探索を行う
    
    for result_com_number in tqdm(result_com_df['result_com'].values):
        try:
            result_com=result_com_number

            #result_comごとの閾値の決定========================================================================
            result_s=result_com_df[result_com_df['result_com']==result_com]
            #===============================================================================
            #学習データのラベル変換==========================================================
            result_train_df=train_df.copy()
            result_train_df=trans_result_com(result_com,result_train_df)

            result_test_year1_df=test_year1_df.copy()
            result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
            result_test_year2_df=test_year2_df.copy()
            result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
            result_test_year3_df=test_year3_df.copy()
            result_test_year3_df=trans_result_com(result_com,result_test_year3_df)#対象のラベルを１、それ以外を０に変換する関数

            result_train_df['money']=train_money
            result_test_year1_df['money']=test_year1_money
            result_test_year2_df['money']=test_year2_money
            result_test_year3_df['money']=test_year3_money
            #学習データラベル変換終わり============================================

            target_per_arr=[500,1000,2000,3000,5000,8000,10000]
            #target_per_arr=[20]
            for sum_target_per in target_per_arr:
                #index=sum_target_per-1
                target_per=100+sum_target_per#学習データを増やす
                #target_per_arr[index]=target_per
                #======================
                #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
                # 一層目の判別機のtrainデータ　:terget_result_df
                target_df=result_train_df.copy()#ベースのデータフレームをコピー
                target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
                target_1_df=target_df[target_df['result_com']==1].copy()
                len_1=len(target_1_df)

                target_0_df=target_df[target_df['result_com']==0].copy()
                len_0=len(target_0_df)
                target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
                target_train_df=pd.concat([target_1_df, target_0_df]).copy()
                #学習＆予測ぱーと========================================================================
                #==========================================================================================================================================
                #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
                target_com_train=target_train_df.drop('money',axis=1).copy()
        #             target_x_train=target_x_train.drop('result_com',axis=1)
        #             target_y_train=target_train_df['result_com'].copy()

                target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
                target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)
                target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
                target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)
                target_x_year3_test=result_test_year3_df.drop('money',axis=1).copy()
                target_x_year3_test=target_x_year3_test.drop('result_com',axis=1)

                target_y_year1_test=result_test_year1_df['result_com'].copy()
                target_y_year2_test=result_test_year2_df['result_com'].copy()
                target_y_year3_test=result_test_year3_df['result_com'].copy()

                #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
                exp = setup(target_com_train,target='result_com',
                            use_gpu=True,
                            train_size=0.7,
                            data_split_shuffle=False,
                            n_jobs=30,
                            silent=True,
                            fix_imbalance=True,
                            session_id=7#rondom_stateみたいなもん，再現性を持たせる
                            #fix_imbalance_method=ADASYN(sampling_strategy='minority')
                           )#学習環境の作成

                #compare_models( errors="raise")
                model=create_model('rf')#モデルの生成（Extra Trees Regressor）
                #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
                tuned_model=tune_model(model,optimize ='F1',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う

                #predict_model(tuned_model)
                # 未知データに対する予測値
                predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
                predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
                predict_year3_df = predict_model(tuned_model, data=target_x_year3_test)

                #==========================================================================================================================================
                #[1]の正答率を見る
                pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year1_test})
                pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year2_test})
                pred_year3_test_df=pd.DataFrame({'pred':predict_year3_df['Label'].values#確率分布での出力
                                                 , 'trans_result':target_y_year3_test})
                year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
                year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
                year3_trans_df=pred_year3_test_df.copy()#閾値での変換は行わない
                #/////収益計算の項
                year1_trans_df['money']=test_year1_money
                year1_trans_df['true_result']=test_year1_df['result_com']
                year2_trans_df['money']=test_year2_money
                year2_trans_df['true_result']=test_year2_df['result_com']
                year3_trans_df['money']=test_year3_money
                year3_trans_df['true_result']=test_year3_df['result_com']
                #配当金の情報も考慮する。
                #result_gain_base_df=calc_gain(trans_df)
                year1_result_gain_base_df=calc_gain(year1_trans_df)
                year2_result_gain_base_df=calc_gain(year2_trans_df)
                year3_result_gain_base_df=calc_gain(year3_trans_df)

                #scoreのseriesに情報書き込み==================
                model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')

                model_score_s['target_com']=result_com#目標としているresult_comラベル番号
                model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
                #model_score_s['threshold']=th

                result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df,year3_result_gain_base_df]
                year_labels=[1,2,3]
                #年のごとのスコア情報を横に展開していく
                for year_df,label in zip(result_gain_df_arr,year_labels):
                    if label !=3:
                        #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                        model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                        #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                        model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                        #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                        model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                        #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                        model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                        #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                        model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                        #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                        model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                        #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                        model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
                    else:
                        model_score_s['gain_year{year}'.format(year=label)]=(year_df["gain"].sum()/(100*year_df["pred"].sum()))*100
                model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
                #モデルの「スコアを保存
                dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/5950_num_{place_name}_model_score_{V}_test_imb_rf.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
                model_score_df.to_csv(dir_path, encoding='utf_8_sig')#定期的に吐き出す方針に変更した
        except:
            print('Error')
    return None


In [ ]:
#rf,不均衡データ版
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_2'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2018,2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
#result_df=result_df.drop('date',axis=1)
model_score_number_pc_rf_year_imb_3years(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9369,0.7132,0.4000,0.1935,0.2609,0.2321,0.2489
1,0.9258,0.7542,0.3333,0.1429,0.2000,0.1676,0.1843
2,0.8924,0.6844,0.3333,0.0943,0.1471,0.1084,0.1335
3,0.9165,0.7753,0.1333,0.0588,0.0816,0.0447,0.0489
4,0.9202,0.6930,0.2000,0.0882,0.1224,0.0872,0.0953
5,0.9147,0.7425,0.2000,0.0811,0.1154,0.0789,0.0879
6,0.9165,0.6604,0.2667,0.1053,0.1509,0.1157,0.1297
7,0.9164,0.7884,0.2667,0.1053,0.1509,0.1156,0.1296
8,0.9368,0.7336,0.2143,0.1154,0.1500,0.1202,0.1265
9,0.9071,0.6696,0.1429,0.0500,0.0741,0.0369,0.0427


In [8]:
#gbc,不均衡データ版
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_2'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2018,2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
#result_df=result_df.drop('date',axis=1)
model_score_number_pc_gbc_year_imb_3years(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9206,0.6172,0.2222,0.0488,0.0800,0.0559,0.0742
1,0.9223,0.7209,0.1250,0.0256,0.0426,0.0201,0.0272
2,0.9361,0.7522,0.1250,0.0323,0.0513,0.0300,0.0376
3,0.9481,0.8167,0.2500,0.0769,0.1176,0.0986,0.1172
4,0.9170,0.6511,0.1250,0.0238,0.0400,0.0171,0.0239
5,0.9239,0.7987,0.3750,0.0714,0.1200,0.0991,0.1380
6,0.9256,0.6434,0.1250,0.0270,0.0444,0.0222,0.0295
7,0.9256,0.7305,0.1250,0.0270,0.0444,0.0222,0.0295
8,0.9187,0.6971,0.2500,0.0465,0.0784,0.0564,0.0793
9,0.9221,0.7246,0.1250,0.0256,0.0426,0.0200,0.0272


100%|█████████████████████████████████████████████████████████████████████████████| 30/30 [43:02:41<00:00, 5165.40s/it]

DONE


In [ ]:
#ada,不均衡データ版
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_2'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2018,2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
#result_df=result_df.drop('date',axis=1)
model_score_number_pc_ada_year_imb_3years(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9416,0.6065,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.9416,0.6428,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.9426,0.6242,0.0169,0.5000,0.0328,0.0291,0.0840
3,0.9377,0.6915,0.0000,0.0000,0.0000,-0.0091,-0.0173
4,0.9416,0.6584,0.0000,0.0000,0.0000,-0.0019,-0.0077
5,0.9377,0.5607,0.0000,0.0000,0.0000,-0.0091,-0.0173
6,0.9426,0.6629,0.0169,0.5000,0.0328,0.0291,0.0840
7,0.9426,0.6871,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.9376,0.6086,0.0000,0.0000,0.0000,-0.0091,-0.0173
9,0.9415,0.5674,0.0000,0.0000,0.0000,-0.0019,-0.0077


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:06:04
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Ada Boost Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:   34.6s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:  8.6min
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed: 18.9min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed: 39.1min


In [ ]:
#rf,不均衡データ版
#設定する変数=================================================================
version_big='v5'#バージョン(大のくくり)
version='v5_2'#バージョン

place_master=master.get_place_master()
place_name='asiya'
#設定する変数=================================================================


result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
result_base_df=pd.read_csv(result_filepath)
result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



years=[2018,2019,2020]
result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
#result_df=result_df.drop('date',axis=1)
model_score_number_pc_rf_year_imb_3years_5950(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8849,0.6329,0.0000,0.0000,0.0000,-0.0031,-0.0142
1,0.8864,0.7022,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.8849,0.5609,0.0000,0.0000,0.0000,-0.0031,-0.0142
3,0.8864,0.6118,0.0139,0.5000,0.0270,0.0210,0.0685
4,0.8849,0.6403,0.0000,0.0000,0.0000,-0.0031,-0.0142
5,0.8833,0.6418,0.0000,0.0000,0.0000,-0.0062,-0.0201
6,0.8864,0.6057,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.8864,0.6024,0.0139,0.5000,0.0270,0.0210,0.0685
8,0.8880,0.6372,0.0139,1.0000,0.0274,0.0244,0.1110
9,0.8815,0.6340,0.0000,0.0000,0.0000,-0.0121,-0.0283


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,08:57:18
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=30)]: Using backend LokyBackend with 30 concurrent workers.
[Parallel(n_jobs=30)]: Done 140 tasks      | elapsed:  2.9min
[Parallel(n_jobs=30)]: Done 390 tasks      | elapsed:  5.8min
[Parallel(n_jobs=30)]: Done 740 tasks      | elapsed:  7.7min


# 関数作成用

### うまく動いているのか確認を行う

In [8]:
def model_score_number_pc_et_year_imb(version,years,place_name,result_df):#pycaretを使ってモデリング(et),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    #result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,20,50,100,500,1000,3000,10000]
        #target_per_arr=[100000]

        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            exp = setup(target_com_train,target='result_com',
                        use_gpu=True,
                        train_size=0.99999,
                        data_split_shuffle=False,
                        n_jobs=5,
                        silent=True,
                        fix_imbalance=True,
                        fix_imbalance_method=ADASYN()
                       )#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('rf')#モデルの生成（Extra Trees Regressor）
            #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            tuned_model=tune_model(model,optimize ='F1',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う

            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
            predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_{place_name}_model_score_{V}_test_imb_rf.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

def model_score_number_nb_et_year_imb(version,years,place_name,result_df):#pycaretを使ってモデリング(et),年単位で検証
    print(place_name)
    #result_dfは加工関数にて分けられたものを渡す。
    model_score_df=pd.DataFrame(columns=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf
    year1=years[0]
    year2=years[1]
    test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
    test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

    train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
    #学習データを切り分けたらyearはいらないから削除する
    test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
    test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

    train_df=train_df.drop(['year'],axis=1).copy()
    #金額の情報は横によけておく
    test_year1_money=pd.Series(test_year1_df['money']).copy()
    test_year2_money=pd.Series(test_year2_df['money']).copy()
    train_money=pd.Series(train_df['money']).copy()

    #出現数の分布
    result_com_s=train_df['result_com'].value_counts()
    result_com_s=result_com_s.sort_index()
    result_com_df=pd.DataFrame({'result_com':result_com_s.index})
    #result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

    for result_com_number in tqdm(result_com_df['result_com'].values):
        result_com=result_com_number

        #result_comごとの閾値の決定========================================================================
        result_s=result_com_df[result_com_df['result_com']==result_com]
        #===============================================================================
        #学習データのラベル変換==========================================================
        result_train_df=train_df.copy()
        result_train_df=trans_result_com(result_com,result_train_df)
        result_test_year1_df=test_year1_df.copy()
        result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
        result_test_year2_df=test_year2_df.copy()
        result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
        result_train_df['money']=train_money
        result_test_year1_df['money']=test_year1_money
        result_test_year2_df['money']=test_year2_money
        #学習データラベル変換終わり============================================

        target_per_arr=[5,20,50,100,500,1000,3000,10000]
        #target_per_arr=[100000]

        #target_per_arr=[20]
        for sum_target_per in target_per_arr:
            #index=sum_target_per-1
            target_per=100+sum_target_per#学習データを増やす
            #target_per_arr[index]=target_per
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df.copy()#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=7)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1].copy()
            len_1=len(target_1_df)

            target_0_df=target_df[target_df['result_com']==0].copy()
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df]).copy()
            #学習＆予測ぱーと========================================================================
            #==========================================================================================================================================
            #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
            target_com_train=target_train_df.drop('money',axis=1).copy()
    #             target_x_train=target_x_train.drop('result_com',axis=1)
    #             target_y_train=target_train_df['result_com'].copy()

            target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
            target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

            target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
            target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

            target_y_year1_test=result_test_year1_df['result_com'].copy()
            target_y_year2_test=result_test_year2_df['result_com'].copy()

            #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
            exp = setup(target_com_train,target='result_com',
                        use_gpu=True,
                        train_size=0.99999,
                        data_split_shuffle=False,
                        n_jobs=5,
                        silent=True,
                        fix_imbalance=True,
                        fix_imbalance_method=ADASYN()
                       )#学習環境の作成

            #compare_models( errors="raise")
            model=create_model('nb')#モデルの生成（Extra Trees Regressor）
            #tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
            tuned_model=tune_model(model,optimize ='F1',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う

            #predict_model(tuned_model)
            # 未知データに対する予測値
            predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
            predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
            #==========================================================================================================================================
            #[1]の正答率を見る
            pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year1_test})
            pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
                                             , 'trans_result':target_y_year2_test})
            year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
            year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
            #/////収益計算の項
            year1_trans_df['money']=test_year1_money
            year1_trans_df['true_result']=test_year1_df['result_com']
            year2_trans_df['money']=test_year2_money
            year2_trans_df['true_result']=test_year2_df['result_com']
            #配当金の情報も考慮する。
            #result_gain_base_df=calc_gain(trans_df)
            year1_result_gain_base_df=calc_gain(year1_trans_df)
            year2_result_gain_base_df=calc_gain(year2_trans_df)

            #scoreのseriesに情報書き込み==================
            model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #model_score_s['threshold']=th

            result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
            year_labels=[1,2]
            #年のごとのスコア情報を横に展開していく
            for year_df,label in zip(result_gain_df_arr,year_labels):
                #model_score_s['総収益']=result_gain_base_df["gain"].sum()
                model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
                #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
                model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
                #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
                model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
                #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
                model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
                #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
                model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
                #model_score_s['的中数']=result_gain_base_df['hit'].sum()
                model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
                #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
                model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
            model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
    #モデルの「スコアを保存
    dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/num_{place_name}_model_score_{V}_test_imb_nb.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
    model_score_df.to_csv(dir_path, encoding='utf_8_sig')
    return None

In [12]:

# #knn版
# #設定する変数=================================================================
# version_big='v5'#バージョン(大のくくり)
# version='v5_2'#バージョン

# place_master=master.get_place_master()
# place_name='asiya'
# #設定する変数=================================================================


# result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
# result_base_df=pd.read_csv(result_filepath)
# result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



# years=[2019,2020]
# result_df=data_making_mo_bo(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
# result_df=result_df.drop('date',axis=1)
# model_score_number_pc_et_year_check(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証
# print('DONE')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9300,0.6620,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.9300,0.6935,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.9288,0.6576,0.0000,0.0000,0.0000,-0.0023,-0.0095
3,0.9299,0.6433,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.9299,0.6777,0.0085,0.5000,0.0167,0.0144,0.0581
5,0.9293,0.5983,0.0000,0.0000,0.0000,-0.0012,-0.0067
6,0.9299,0.6463,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.9299,0.6693,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.9287,0.5795,0.0000,0.0000,0.0000,-0.0023,-0.0095
9,0.9299,0.6241,0.0000,0.0000,0.0000,-0.0012,-0.0067


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:35:11
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
  0%|                                                                                                                       | 0/28 [04:26<?, ?it/s]


KeyboardInterrupt: 

In [72]:
# #設定する変数=================================================================
# version='V5'#バージョン

# place_master=master.get_place_master()
# place_name='asiya'
# #設定する変数=================================================================


# result_filepath="../../../bot_database/{place_name}/{place_name}_train/train_{place_name}.csv".format(place_name=place_name)#作成したデータの書き込み先#使用するデータの読み込み
# result_base_df=pd.read_csv(result_filepath)
# result_base_df=result_base_df.drop(["Unnamed: 0"],axis=1)



# years=[2019,2020]
# result_df=data_making_none_clustar(result_base_df)#モデル関連に使用するdfの作成関数，要は前処理を一気にやってくれる関数(クラスタリング無し、モータ番号、艇番号なし)
# result_df=result_df.drop('date',axis=1)
# #model_score_pc_et_year(version,years,place_name,result_df)#pycaretを使ってモデリング(et),年単位で検証









# print(place_name)
# #result_dfは加工関数にて分けられたものを渡す。
# model_score_df=pd.DataFrame(columns=['target_com','target_per','threshold','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'])#スコアを格納するdf

# year1=years[0]
# year2=years[1]
# test_year1_df= result_df[(result_df['year']==year1)].copy()#2019のデータ
# test_year2_df= result_df[(result_df['year']==year2)].copy()#2020のデータ

# train_df =  result_df[(result_df['year']!=year1) & (result_df['year']!=year2)].copy()#そのほかを学習データに
# #学習データを切り分けたらyearはいらないから削除する
# test_year1_df=test_year1_df.drop(['year'],axis=1).copy()
# test_year2_df=test_year2_df.drop(['year'],axis=1).copy()

# train_df=train_df.drop(['year'],axis=1).copy()
# #金額の情報は横によけておく
# test_year1_money=pd.Series(test_year1_df['money']).copy()
# test_year2_money=pd.Series(test_year2_df['money']).copy()
# train_money=pd.Series(train_df['money']).copy()

# #出現数の分布
# result_com_s=train_df['result_com'].value_counts()
# result_com_s=result_com_s.sort_index()
# result_com_df=pd.DataFrame({'result_com':result_com_s.index})
# result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため

# for result_com_number in tqdm(result_com_df['result_com'].values):
#     result_com=result_com_number

#     #result_comごとの閾値の決定========================================================================
#     result_s=result_com_df[result_com_df['result_com']==result_com]
#     #===============================================================================
#     #学習データのラベル変換==========================================================
#     result_train_df=train_df.copy()
#     result_train_df=trans_result_com(result_com,result_train_df)
#     result_test_year1_df=test_year1_df.copy()
#     result_test_year1_df=trans_result_com(result_com,result_test_year1_df)#対象のラベルを１、それ以外を０に変換する関数
#     result_test_year2_df=test_year2_df.copy()
#     result_test_year2_df=trans_result_com(result_com,result_test_year2_df)#対象のラベルを１、それ以外を０に変換する関数
#     result_train_df['money']=train_money
#     result_test_year1_df['money']=test_year1_money
#     result_test_year2_df['money']=test_year2_money
#     #学習データラベル変換終わり============================================

#     #target_per_arr=[5,10,25,50,100,200,500]
#     target_per_arr=[20]
#     for sum_target_per in target_per_arr:
#         #index=sum_target_per-1
#         target_per=100+sum_target_per#学習データを増やす
#         #target_per_arr[index]=target_per
#         #======================
#         #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
#         # 一層目の判別機のtrainデータ　:terget_result_df
#         target_df=result_train_df.copy()#ベースのデータフレームをコピー
#         target_1_df=target_df[target_df['result_com']==1].copy()
#         len_1=len(target_1_df)
        
#         target_0_df=target_df[target_df['result_com']==0].copy()
#         len_0=len(target_0_df)
#         target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0].copy()#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
#         target_train_df=pd.concat([target_1_df, target_0_df]).copy()
#         #学習＆予測ぱーと========================================================================
#         #==========================================================================================================================================
#         #データの切り分け(pycaretは引数でyを指定するので，予測対象のresult_comは分別する必要はない，収益moneyは予測の変数として　用意できないので消す.)
#         target_com_train=target_train_df.drop('money',axis=1).copy()
# #             target_x_train=target_x_train.drop('result_com',axis=1)
# #             target_y_train=target_train_df['result_com'].copy()

#         target_x_year1_test=result_test_year1_df.drop('money',axis=1).copy()
#         target_x_year1_test=target_x_year1_test.drop('result_com',axis=1)

#         target_x_year2_test=result_test_year2_df.drop('money',axis=1).copy()
#         target_x_year2_test=target_x_year2_test.drop('result_com',axis=1)

#         target_y_year1_test=result_test_year1_df['result_com'].copy()
#         target_y_year2_test=result_test_year2_df['result_com'].copy()

#         #モデリングを行う，行う対象はtargetのcomを1,0でラベル変換したdfに対して．
#         exp = setup(target_com_train,target='result_com',use_gpu=True,train_size=0.99999,data_split_shuffle=False,n_jobs=10)#学習環境の作成
#         #compare_models( errors="raise")
#         model=create_model('rf')#モデルの生成（Extra Trees Regressor）
#         tuned_model=tune_model(model,optimize ='Precision',n_iter = 100)#生成したモデルをもとにハイパーパラメータチューニングを行う
#         #predict_model(tuned_model)
#         # 未知データに対する予測値
#         predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
#         predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
#         #==========================================================================================================================================
#         #[1]の正答率を見る
#         pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
#                                          , 'trans_result':target_y_year1_test})
#         pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
#                                          , 'trans_result':target_y_year2_test})
#         year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
#         year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
#         #/////収益計算の項
#         year1_trans_df['money']=test_year1_money
#         year1_trans_df['true_result']=test_year1_df['result_com']
#         year2_trans_df['money']=test_year2_money
#         year2_trans_df['true_result']=test_year2_df['result_com']
#         #配当金の情報も考慮する。
#         #result_gain_base_df=calc_gain(trans_df)
#         year1_result_gain_base_df=calc_gain(year1_trans_df)
#         year2_result_gain_base_df=calc_gain(year2_trans_df)

#         #scoreのseriesに情報書き込み==================
#         model_score_s=pd.Series(index=['target_com','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
#         model_score_s['target_com']=result_com#目標としているresult_comラベル番号
#         model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
#         #model_score_s['threshold']=th

#         result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
#         year_labels=[1,2]
#         #年のごとのスコア情報を横に展開していく
#         for year_df,label in zip(result_gain_df_arr,year_labels):
#             #model_score_s['総収益']=result_gain_base_df["gain"].sum()
#             model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
#             #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
#             model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
#             #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
#             model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
#             #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
#             model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
#             #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
#             model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
#             #model_score_s['的中数']=result_gain_base_df['hit'].sum()
#             model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
#             #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
#             model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
#         model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
#     #モデルの「スコアを保存
#     dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
#     model_score_df.to_csv(dir_path, encoding='utf_8_sig')
# #return None

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.6462,0.7943,0.2203,1.0000,0.3611,0.2359,0.3657
1,0.6154,0.7641,0.1610,0.9500,0.2754,0.1656,0.2877
2,0.6178,0.7786,0.1695,0.9524,0.2878,0.1741,0.2963
3,0.6062,0.7644,0.1780,0.8077,0.2917,0.1522,0.2362
4,0.5869,0.7678,0.1271,0.7895,0.2190,0.1060,0.1886
5,0.5869,0.7532,0.1102,0.8667,0.1955,0.1033,0.2047
6,0.5830,0.7016,0.1356,0.7273,0.2286,0.0997,0.1662
7,0.5174,0.4925,0.1610,0.4222,0.2331,-0.0246,-0.0307
8,0.5521,0.4783,0.0763,0.5625,0.1343,0.0286,0.0551
9,0.5212,0.4801,0.0855,0.3704,0.1389,-0.0367,-0.0558


  4%|██▉                                                                                | 1/28 [01:12<32:24, 72.02s/it]

IntProgress(value=0, description='Processing: ', max=3)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,09:31:22
Status,. . . . . . . . . . . . . . . . . .,Finalizing Model
Estimator,. . . . . . . . . . . . . . . . . .,K Neighbors Classifier


Text(value="Following data types have been inferred automatically, if they are correct press enter to continue…

,Data Type
racer_1_rank,Categorical
racer_1_age,Numeric
racer_1_doub,Numeric
racer_1_ave_st,Numeric
racer_2_rank,Categorical
racer_2_age,Numeric
racer_2_doub,Numeric
racer_2_ave_st,Numeric
racer_3_rank,Categorical
racer_3_age,Numeric


  4%|██▉                                                                               | 1/28 [01:49<49:08, 109.20s/it]


KeyboardInterrupt: Interrupted by user

In [56]:
# # 未知データに対する予測値
# predict_year1_df = predict_model(tuned_model, data=target_x_year1_test)
# predict_year2_df = predict_model(tuned_model, data=target_x_year2_test)
# #==========================================================================================================================================
# #[1]の正答率を見る
# pred_year1_test_df=pd.DataFrame({'pred':predict_year1_df['Label'].values#確率分布での出力
#                                  , 'trans_result':target_y_year1_test})
# pred_year2_test_df=pd.DataFrame({'pred':predict_year2_df['Label'].values#確率分布での出力
#                                  , 'trans_result':target_y_year2_test})
# year1_trans_df=pred_year1_test_df.copy()#閾値での変換は行わない
# year2_trans_df=pred_year2_test_df.copy()#閾値での変換は行わない
# #/////収益計算の項
# year1_trans_df['money']=test_year1_money
# year1_trans_df['true_result']=test_year1_df['result_com']
# year2_trans_df['money']=test_year2_money
# year2_trans_df['true_result']=test_year2_df['result_com']
# #配当金の情報も考慮する。
# #result_gain_base_df=calc_gain(trans_df)
# year1_result_gain_base_df=calc_gain(year1_trans_df)
# year2_result_gain_base_df=calc_gain(year2_trans_df)

# #scoreのseriesに情報書き込み==================
# model_score_s=pd.Series(index=['target_com','depth','target_per','total_get_year1', 'total_use_year1','num_com_year1','num_pred_year1','num_hit_year1','buy_hit_per_year1','gain_year1','total_get_year2', 'total_use_year2','num_com_year2','num_pred_year2','num_hit_year2','buy_hit_per_year2','gain_year2','gain_year3'],dtype='float64')
# model_score_s['target_com']=result_com#目標としているresult_comラベル番号
# model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
# model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
# #model_score_s['threshold']=th

# result_gain_df_arr=[year1_result_gain_base_df,year2_result_gain_base_df]
# year_labels=[1,2]
# #年のごとのスコア情報を横に展開していく
# for year_df,label in zip(result_gain_df_arr,year_labels):
#     #model_score_s['総収益']=result_gain_base_df["gain"].sum()
#     model_score_s['total_get_year{year}'.format(year=label)]=year_df["gain"].sum()
#     #model_score_s['投資金額']=100*result_gain_base_df["pred"].sum()
#     model_score_s['total_use_year{year}'.format(year=label)]=100*year_df["pred"].sum()
#     #model_score_s['出現数']=result_gain_base_df['trans_result'].sum()
#     model_score_s['num_com_year{year}'.format(year=label)]=year_df['trans_result'].sum()
#     #model_score_s['購買予測数']=result_gain_base_df['pred'].sum()
#     model_score_s['num_pred_year{year}'.format(year=label)]=year_df['pred'].sum()
#     #model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
#     model_score_s['gain_year{year}'.format(year=label)]=(model_score_s['total_get_year{year}'.format(year=label)]/model_score_s['total_use_year{year}'.format(year=label)])*100
#     #model_score_s['的中数']=result_gain_base_df['hit'].sum()
#     model_score_s['num_hit_year{year}'.format(year=label)]=year_df['hit'].sum()
#     #model_score_s['購買的中率']=(model_score_s['的中数']/ model_score_s['購買予測数'])*100
#     model_score_s['buy_hit_per_year{year}'.format(year=label)]=(model_score_s['num_hit_year{year}'.format(year=label)]/ model_score_s['num_pred_year{year}'.format(year=label)])*100
# #             model_score_df=model_score_df.append(model_score_s,ignore_index=True,sort=False)
# #     #モデルの「スコアを保存
# #     dir_path =  "../../bot_database/{place_name}/model_score_{place_name}/{place_name}_model_score_{V}.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
# #     model_score_df.to_csv(dir_path, encoding='utf_8_sig')

AttributeError: 'DataTypes_Auto_infer' object has no attribute 'final_training_columns'

In [48]:
# compare_models( errors="raise")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7401,0.7572,0.4435,0.6639,0.5223,0.3583,0.3752,0.5480
lda,Linear Discriminant Analysis,0.7348,0.7497,0.4625,0.6357,0.5276,0.3537,0.3653,0.0180
xgboost,Extreme Gradient Boosting,0.7333,0.7549,0.5276,0.6121,0.5588,0.3726,0.3784,0.6400
lightgbm,Light Gradient Boosting Machine,0.7330,0.7587,0.5018,0.6139,0.5451,0.3629,0.3693,0.0680
et,Extra Trees Classifier,0.7263,0.7493,0.3450,0.6501,0.4430,0.2928,0.3177,0.2700
rf,Random Forest Classifier,0.7203,0.7468,0.3315,0.6655,0.4329,0.2787,0.3110,0.2600
ada,Ada Boost Classifier,0.7188,0.7325,0.4817,0.5880,0.5255,0.3311,0.3358,0.1570
ridge,Ridge Classifier,0.7139,0.0000,0.3495,0.6371,0.4453,0.2758,0.3007,0.0080
lr,Logistic Regression,0.6911,0.7084,0.3092,0.5834,0.3997,0.2173,0.2396,0.3100
dummy,Dummy Classifier,0.6665,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0050


GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=2772, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [52]:
target_x_year1_test

,racer_1_rank,racer_1_age,racer_1_doub,racer_1_ave_st,racer_2_rank,racer_2_age,racer_2_doub,racer_2_ave_st,racer_3_rank,racer_3_age,...,racer_2_male_0,racer_2_male_1,racer_3_male_0,racer_3_male_1,racer_4_male_0,racer_4_male_1,racer_5_male_0,racer_5_male_1,racer_6_male_0,racer_6_male_1
15171,4,31.0,0.453,0.16,1,27.0,0.170,0.16,2,51.0,...,1,0,1,0,1,0,1,0,1,0
15172,3,52.0,0.328,0.18,1,34.0,0.121,0.21,4,23.0,...,1,0,1,0,1,0,1,0,1,0
15173,4,38.0,0.525,0.16,2,57.0,0.078,0.22,4,34.0,...,1,0,1,0,1,0,1,0,1,0
15174,3,31.0,0.344,0.19,4,38.0,0.450,0.18,2,34.0,...,1,0,1,0,1,0,1,0,1,0
15175,2,32.0,0.245,0.20,3,47.0,0.388,0.15,3,32.0,...,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17285,2,45.0,0.234,0.16,2,39.0,0.250,0.18,1,45.0,...,0,1,0,1,0,1,0,1,0,1
17286,3,34.0,0.404,0.18,2,31.0,0.279,0.16,2,54.0,...,0,1,0,1,0,1,0,1,0,1
17287,3,32.0,0.398,0.19,3,38.0,0.349,0.18,3,44.0,...,0,1,0,1,0,1,0,1,0,1
17288,4,32.0,0.483,0.16,3,52.0,0.426,0.17,3,45.0,...,0,1,0,1,0,1,0,1,0,1


In [75]:
dir_path =  "../../../bot_database/{place_name}/model_score_{place_name}/v5_score/{place_name}_model_score_{V}_test.csv".format(place_name=place_name,V=version)#作成したデータの書き込み先
model_score_df.to_csv(dir_path, encoding='utf_8_sig')

In [68]:
pred_year1_test_df.to_csv("ex_year1.csv")
pred_year2_test_df.to_csv("ex_year2.csv")

In [60]:
tuned_model


RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight={},
                       criterion='entropy', max_depth=11, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.002, min_impurity_split=None,
                       min_samples_leaf=6, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=10, oob_score=False, random_state=3514, verbose=0,
                       warm_start=False)